<a href="https://colab.research.google.com/github/PalakGupta5/Data_Science-Hackathon/blob/master/Hackathon_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [0]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import sys
from google.colab import files
import csv

### Intsall Selenium on Colab

In [0]:
'''
#install selenium on colab
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

'''

"\n#install selenium on colab\n!pip install selenium\n!apt-get update # to update ubuntu to correctly run apt install\n!apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\n\nfrom selenium import webdriver\nsys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')\n\nchrome_options = webdriver.ChromeOptions()\nchrome_options.add_argument('--headless')\nchrome_options.add_argument('--no-sandbox')\nchrome_options.add_argument('--disable-dev-shm-usage')\nbrowser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)\n\n"

### Urls to scrape the data

In [0]:
#urls
main_url='https://www.prokabaddi.com'
stats_all = 'https://www.prokabaddi.com/stats'
url_alt = 'https://www.sportskeeda.com/go/pro-kabaddi/stats'

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}

In [0]:
#Get the details from the main page of Pro kabaddi.

response = requests.get(main_url, headers=headers)
print(response)

<Response [200]>


### Extract links for all the team from main page of pro kabaddi and store into a csv file: 'team_links.csv' on local machine


In [0]:
soup = bs(response.text, 'html.parser')
df_team_links = pd.DataFrame()
links = soup.find_all('a')

team_links = [main_url + x.attrs['href'] for x in links[13:25]]
team_name = [x.text for x in links[13:25]]
df_team_links['team_name'] = team_name
df_team_links['team_url_link'] = team_links
df_team_links

with open('team_links.csv', 'w') as f:
  employee_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  
  for each in range(len(df_team_links)):
    row1 = df_team_links.iloc[each].values.flatten().tolist()
    #print(row1)
    employee_writer.writerow(row1)
    
files.download('team_links.csv')

### Extract team stats for each team and for every seasons and store into CSV file

In [0]:
#use selenium
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

ext_data = []
i = 0
for m in team_links:
  d.get(m)
  time.sleep(5)
  d.maximize_window()
  html = d.page_source

  soup_1 = bs(html, 'html.parser')
  cols  = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-tbl-header'}).find_all('div', {'class': 'si-data-block'})
  main_cols = [x.text for x in cols]
  cols1 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-overall'}).find_all('div', {'class': 'si-data-block'})
  cols_x = [x.text for x in cols1]
  cols2 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-attack'}).find_all('div', {'class': 'si-data-block'})
  cols2_l = [x.text for x in cols2]
  cols3 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-defence'}).find_all('div', {'class': 'si-data-block'})
  cols3_l = [x.text for x in cols3]
  main_cols = main_cols + cols_x + cols2_l + cols3_l
  team_cols = ['team_name']
  main_cols = team_cols + main_cols
  
  datahead = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-right'}).find('div', {'class': 'si-tbl-header'}).find_all('div', {'class': 'si-data-block'})
  datahead_t = [x.text for x in datahead]

  data = []
  for each1 in range(1,4):
    list1 = []
    for each in range(len(datahead_t)):
      datahead1 = soup_1.find('div', {'class': 'si-team-stats'}).find('div', {'class': 'si-right'}).find_all('div', {'class': 'si-tbl-wraper'})[each1].find_all('div', {'class': 'si-tbl-data'})[each].find_all('div', {'class': 'si-data-block'})
      datahead1_t = [x.text.strip('\n').split('\n')[0] for x in datahead1]
      list1.append(datahead1_t)
    data.append(list1)

  newdata=[]
  for each in range(len(data[0])):
    newdata.append(data[0][each] + data[1][each] + data[2][each])

  newdata1=[]
  for each in range(len(newdata)):
    temp = []
    temp.append(team_name[i])
    temp.append(datahead_t[each])
    for inside in range(len(newdata[0])):
      temp.append(newdata[each][inside])
    newdata1.append(temp)
  
  ext_data.append(newdata1)
  i = i + 1


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chrom

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


[[['Bengal Warriors',
   'Overall',
   '124',
   '55',
   '15',
   '54',
   '-',
   '48',
   '31',
   '5269',
   '1764',
   '1143',
   '2362',
   '42%',
   '70',
   ' 1725 ',
   ' 483',
   '2208',
   '2812',
   '1017',
   '1795',
   ' 39%',
   ' 98',
   ' 130',
   ' 260',
   ' 1110'],
  ['Bengal Warriors',
   'Season 7',
   '19',
   '12',
   '3',
   '4',
   '1',
   '48',
   '31',
   '782',
   '318',
   '153',
   '311',
   '51%',
   '15',
   ' 322 ',
   ' 76',
   '398',
   '449',
   '174',
   '275',
   ' 41%',
   ' 12',
   ' 31',
   ' 62',
   ' 184'],
  ['Bengal Warriors',
   'Season 6',
   '23',
   '12',
   '2',
   '9',
   '2',
   '44',
   '16',
   '963',
   '340',
   '201',
   '422',
   '45%',
   '16',
   ' 341 ',
   ' 95',
   '436',
   '508',
   '186',
   '322',
   ' 39%',
   ' 16',
   ' 23',
   ' 46',
   ' 200'],
  ['Bengal Warriors',
   'Season 5',
   '24',
   '11',
   '6',
   '7',
   '1',
   '44',
   '20',
   '1079',
   '379',
   '223',
   '477',
   '42%',
   '11',
   ' 333 ',
   

In [0]:
ext_data1 = []
for each in ext_data:
  ext_data1 = ext_data1 + each
  
df_matches_det = pd.DataFrame(data=ext_data1, columns=main_cols)
df_matches_det.head()

with open('team_stats.csv', 'w') as f:
  employee_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  
  for each in range(len(df_matches_det)):
    row1 = df_matches_det.iloc[each].values.flatten().tolist()
    #print(row1)
    employee_writer.writerow(row1)
    
files.download('team_stats.csv')

### Get All schedules and results of every match of Pro Kabaddi for all seasons

In [0]:
divhead = soup.find('div', {'class': 'nav-middle'})
divli = divhead.find('li', {'class': 'schedule-fixtures-results'}).find('a').attrs['href']
schedule_link = main_url + divli


In [0]:
#use selenium
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from selenium.webdriver.support.ui import Select

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

d.get(schedule_link)
time.sleep(5)
d.maximize_window()
html = d.page_source

select = Select(d.find_element_by_class_name('si-season-change'))

data=[]
cols = ['Season', 'Date', 'Time', 'Status', 'Team1', 'Score1', 'Team2', 'Score2', 'Venue', 'Match-Results']

for each_indx in range(1,7):
  select.select_by_index(each_indx)
  time.sleep(5)
  soup_1 = bs(d.page_source)
  div_1= soup_1.find_all('div', {'class':'si-fix-box'})
  
  for each in div_1:
    list1=[]
    if each_indx==1:
      list1.append('Season 6')
    elif each_indx==2:
      list1.append('Season 5')
    elif each_indx==2:
      list1.append('Season 4')
    elif each_indx==2:
      list1.append('Season 3')
    elif each_indx==2:
      list1.append('Season 2')
    else:
      list1.append('Season 1')
    
    list1.append(each.find('div', {'class':'matDate'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'matTime'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'matStatus'}).text.strip('\n'))
    list1.append("".join(each.find_all('div', {'class':'fix-teamName'})[0].find('div', {'class':'si-fullName'}).text.split('\n')))
    list1.append(each.find('div', {'class':'matScore'}).find_all('span', {'class': 'si-score'})[0].text.strip())
    list1.append("".join(each.find_all('div', {'class':'fix-teamName'})[1].find('div', {'class':'si-fullName'}).text.split('\n')))
    list1.append(each.find('div', {'class':'matScore'}).find_all('span', {'class': 'si-score'})[1].text.strip())
    list1.append(each.find('div', {'class':'mat-venue'}).text.strip('\n'))
    list1.append(each.find('div', {'class':'mat-resultStatus'}).text.strip('\n'))
    data.append(list1)

match_res = pd.DataFrame(data=data, columns=cols)
match_res.shape

     |████████████████████████████████| 911kB 5.1MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


(515, 10)

In [0]:
with open('match_res.csv', 'w') as f:
  employee_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  
  for each in range(len(match_res)):
    row1 = match_res.iloc[each].values.flatten().tolist()
    #print(row1)
    employee_writer.writerow(row1)
    
files.download('match_res.csv')

### Extract Player stats from Pro Kabaddi

In [0]:
#use selenium
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


d.get(stats_all)
time.sleep(5)
d.maximize_window()
html = d.page_source


soup = bs(html, 'html.parser')
#Player stats links with respect to its matrics
playerstat_div = soup.find_all('div', {'class': 'si-leadBoard-box'})
metrics = [each.text.split('\n')[3] for each in playerstat_div]

player_stats_urls = []
for each in playerstat_div:
  player_stat = each.find('div', {'class': 'si-leadBoard-viewMore'})
  player_stats_urls.append(main_url + player_stat.find('a').attrs['href'])

player_stats_det = pd.DataFrame()
player_stats_det['player_metrics'] = metrics
player_stats_det['player_stats_url'] = player_stats_urls
player_stats_det



Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chrom

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


,player_metrics,player_stats_url
0,Total Points,https://www.prokabaddi.com/stats/49-102-total-...
1,Raid Points,https://www.prokabaddi.com/stats/49-22-raid-po...
2,Tackle Points,https://www.prokabaddi.com/stats/49-103-tackle...
3,Successful Raids,https://www.prokabaddi.com/stats/49-21-success...
4,Super Raids,https://www.prokabaddi.com/stats/49-104-super-...
5,Super 10s,https://www.prokabaddi.com/stats/49-100-super-...
6,Successful Tackles,https://www.prokabaddi.com/stats/49-23-success...
7,Super Tackles,https://www.prokabaddi.com/stats/49-28-super-t...
8,High 5s,https://www.prokabaddi.com/stats/49-101-high-5...


In [0]:
with open('player_stats_urls.csv', 'w') as f:
  employee_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  
  for each in range(player_stats_det.shape[0]):
    row1 = player_stats_det.iloc[each].values.flatten().tolist()
    #print(row1)
    employee_writer.writerow(row1)
    
files.download('player_stats_urls.csv')

In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import ElementNotInteractableException
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


main_data = []
p = 0
for each_url in player_stats_det['player_stats_url']:
  print(each_url)
  d.get(each_url)
  time.sleep(5)
  d.maximize_window()
  element = d.find_element_by_id('player_Btn')
  element.click()
  time.sleep(5)

  select = Select(d.find_element_by_id('season_dropdown'))
  
  options_det = [o.text for o in select.options]
  data1 = []
  for each_index in range(1, len(options_det)):
    print(options_det[each_index])
    select.select_by_index(each_index)

    time.sleep(5)

    load_more = d.find_element_by_id('load_more')
    
    while load_more:
        try:
            load_more.click()
            time.sleep(5)
            d.maximize_window()
            load_more = d.find_element_by_id('load_more')
        except ElementNotInteractableException:
            break

    soup_1 = bs(d.page_source)
    
    #player_cols_div = soup_1.find('div',attrs={'class':'si-leadBoard-detail-tRow'})
    #player_cols_div_det = player_cols_div.find_all('div',attrs={'class':'si-leadBoard-detail-tCol'})
    #print(len(player_cols_div_det))
    #newcols = [each.text.split('\n')[2] for each in player_cols_div_det]
    #print(newcols)
    
    player_div = soup_1.findAll('div',attrs={'class':'sipk-lb-detailItem'})

    
    print(len(player_div))
    data = []
    for each in player_div:
      list1 = []
      
      list1.append(options_det[each_index])
      list1.append(player_stats_det['player_metrics'][p])
      
      pdet = each.find('div', {'class':'sipk-lb-playerName'})
      if pdet:
        list1.append(pdet.text.split('\n')[3].strip())
      else:
        list1.append(np.NAN)

      pdet = each.find('a', {'class':'sipk-lb-team'})
      if pdet:
        list1.append(" ".join(pdet.attrs['href'].split('/')[2].split('-')[0:2]))
      else:
        list1.append(np.NAN)

      pdet = each.find('div', {'class':'sipk-lb-matchedPlayed'})
      if pdet:
        list1.append(pdet.text.split('\n')[1])
      else:
        list1.append(np.NAN)

      pdet = each.find('div', {'class':'sipk-lb-raidPoints'})
      if pdet:
        list1.append(pdet.text.split('\n')[1])
      else:
        list1.append(np.NAN)

      pdet = each.find('div', {'class':'sipk-lb-playerprofile'})
      if pdet:
        list1.append(pdet.text.split('\n')[1].strip())
      else:
        list1.append(np.NAN)

      data.append(list1)

    data1.append(data)
  main_data.append(data1)
  p = p + 1
  


In [0]:
main_data_final = []
for x in main_data:
  for y in x:
    for z in y:
      main_data_final.append(z)
    
len(main_data_final)
print(main_data_final[0])
cols = ['Season_Name', 'Metrics', 'Player_Name', 'Team_Name', 'Total_matches_played', 'Points', 'Player_type']
player_stat = pd.DataFrame(data=main_data_final, columns=cols)
player_stat.head()

with open('player_stat.csv', 'w') as f:
  employee_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  
  for each in range(len(player_stat)):
    row1 = player_stat.iloc[each].values.flatten().tolist()
    #print(row1)
    employee_writer.writerow(row1)
    
files.download('player_stat.csv')